In [1]:
using JuMP, Gurobi
using CSV, DataFrames, CategoricalArrays, Plots, Statistics

In [2]:
lnr_conv = IAI.read_json("models/lr_conversion.json")
weights_conv = IAI.get_prediction_weights(lnr_conv)[1];
conv_tfidf_weights = zeros(50)
for (k,v) in weights_conv
    if startswith(String(k), "tfidf_")
        # Extract the number after tfidf_
        i = parse(Int, split(String(k), "_")[2])
        conv_tfidf_weights[i+1] = v
    end
end
conv_const = 45.4847
conv_cpc_weight = 9.58332
conv_region_weights = [0, 0, 0, 0] # USA, A, B, C
conv_match_weights = [117.156, 0, 0] # Broad, exact, phrase
conv_month_weight = 0
conv_weights = vcat([conv_month_weight], conv_region_weights, conv_match_weights, conv_tfidf_weights)
;

In [3]:
lnr_clicks = IAI.read_json("models/lr_clicks.json")
weights_clicks = IAI.get_prediction_weights(lnr_clicks)[1];
clicks_tfidf_weights = zeros(50)
for (k,v) in weights_clicks
    if startswith(String(k), "tfidf_")
        # Extract the number after tfidf_
        i = parse(Int, split(String(k), "_")[2])
        clicks_tfidf_weights[i+1] = v
    end
end
clicks_const = 1.14021
clicks_cpc_weight = 0.142413
clicks_region_weights = [0, -0.65995, -0.504572, 1.88284] # USA, A, B, C
clicks_match_weights = [5.41894, -0.513683, 0] # Broad, exact, phrase
clicks_month_weight = -0.0359214

clicks_weights = vcat([clicks_month_weight], clicks_region_weights, clicks_match_weights, clicks_tfidf_weights)
;

In [4]:
keyword_df = CSV.read("unique_keyword_embeddings.csv", DataFrame)
num_keywords = nrow(keyword_df)

146

In [9]:
months     = 1:12
regions    = ["USA", "A", "B", "C"]
matches    = ["broad match", "exact match", "phrase match"]

combos = collect(Iterators.product(1:146, months, regions, matches))
keyword_idx = [c[1] for c in combos]   # Vector{Int}
month_col   = [c[2] for c in combos]
region_col  = [c[3] for c in combos]
match_col   = [c[4] for c in combos]
result = DataFrame(
    keyword = keyword_df[vec(keyword_idx), "Keyword"],
    month   = vec(month_col),
    region  = vec(region_col),
    match   = vec(match_col),
)
# Region one-hot columns
for r in regions
    col = Symbol("region_", r)
    result[!, col] = Int.(result.region .== r)
end

# Match one-hot columns
for mt in matches
    col = Symbol("match_", replace(mt, " " => "_"))   # replace spaces with _
    result[!, col] = Int.(result.match .== mt)
end
embedding_cols = names(keyword_df, r"tfidf_")
for c in embedding_cols
    result[!, c] = keyword_df[vec(keyword_idx), c]
end
select!(result, Not([:keyword, :region, :match]));

Row,month,region_USA,region_A,region_B,region_C,match_broad_match,match_exact_match,match_phrase_match,tfidf_0,tfidf_1,tfidf_2,tfidf_3,tfidf_4,tfidf_5,tfidf_6,tfidf_7,tfidf_8,tfidf_9,tfidf_10,tfidf_11,tfidf_12,tfidf_13,tfidf_14,tfidf_15,tfidf_16,tfidf_17,tfidf_18,tfidf_19,tfidf_20,tfidf_21,tfidf_22,tfidf_23,tfidf_24,tfidf_25,tfidf_26,tfidf_27,tfidf_28,tfidf_29,tfidf_30,tfidf_31,tfidf_32,tfidf_33,tfidf_34,tfidf_35,tfidf_36,tfidf_37,tfidf_38,tfidf_39,tfidf_40,tfidf_41,tfidf_42,tfidf_43,tfidf_44,tfidf_45,tfidf_46,tfidf_47,tfidf_48,tfidf_49
,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,1,1,0,0,0,1,0,0,0.689735,-0.177445,-0.307057,9.04931e-5,0.131787,0.0520535,-0.0285501,-0.212919,0.0304915,-0.0251922,-0.0342965,-0.00774351,0.0219183,-0.0401749,-0.0625218,-0.115255,-0.0498309,-0.0377962,-0.109429,-0.0764247,-0.0459035,0.107646,-0.0413963,-0.0698828,0.0206885,-0.192912,0.0200845,0.0932728,-0.00211518,0.104317,0.118676,0.100759,-0.104279,0.0318823,0.120562,-0.0288379,0.253279,0.0379153,-0.00261354,-0.0632029,0.0317896,-0.12689,-0.0360121,-0.0746933,-0.00190674,-0.0238104,-0.0453563,0.0118163,0.026581,0.0275363
2,1,1,0,0,0,1,0,0,0.133239,0.538104,0.0243651,0.178385,-0.0341666,0.190108,0.0611806,0.0412863,-0.019873,-0.0365506,-0.0284895,0.0107042,-0.0650972,0.0320112,-0.0308569,-0.0715463,-0.0464475,-0.0537274,-0.00231329,0.0238716,0.0490897,0.0286704,-0.0292614,-0.0316323,0.054194,0.0190414,-0.0538425,0.0251139,-0.0950559,-0.0641693,-0.0224434,-0.109864,0.0148892,-0.0742892,0.0850185,0.00482007,0.0174819,-0.118238,0.007976,0.0560625,0.267157,-0.161144,-0.00326441,-0.0140399,-0.0246997,0.00871945,0.045516,0.0353438,0.077807,0.0049838
3,1,1,0,0,0,1,0,0,0.0914185,0.340657,0.00973417,-0.0472038,-0.053728,-0.0114059,0.00321296,0.00858122,0.135346,-0.0137615,-0.158048,0.240853,0.0130656,-0.0634519,0.0181418,0.126493,-0.0426136,0.0537052,-0.0469296,0.021827,-0.000655033,0.000698797,0.0025878,-0.0193253,0.0305114,0.0146821,0.0245869,-0.0374486,-0.0786022,-0.085899,-0.0306111,0.0787154,0.00251304,0.155599,-0.0338436,-0.260965,0.271736,-0.166786,-0.156838,0.24844,-0.0736973,0.237797,0.0127751,0.0734451,0.0518628,-0.0148913,-0.0010094,-0.00524355,-0.0347266,-0.0363895
4,1,1,0,0,0,1,0,0,0.377125,-0.0815655,-0.0318149,-0.0659879,-0.111267,0.019627,0.0936977,-0.0351366,-0.0562823,0.0615443,0.085318,0.0328723,0.111617,0.0523833,-0.0340039,-0.0122263,-0.008737,-0.167486,-0.0359716,-0.0284492,0.0560482,0.0665659,0.0585979,0.224458,0.165235,0.242532,0.541926,-0.0599857,-0.126568,0.0573902,0.0754209,0.00970831,0.00630717,-0.0976584,0.0738345,0.0101046,-0.0358942,0.0153468,-0.000240831,-0.00311795,-0.0104875,-0.00515859,-0.00152276,-0.00916535,0.0201651,-0.0454238,-0.00922887,-0.011766,-0.000194034,0.00967604
5,1,1,0,0,0,1,0,0,0.500447,-0.1587,-0.16713,-0.0608792,-0.171966,0.0584503,-0.0371784,-0.0162403,-0.0187049,-0.0394425,-0.0649989,0.0471174,-0.0371457,0.0162479,0.00637863,0.0262636,-0.137481,-0.0620277,-0.0562648,0.0110616,0.0365562,-0.0287666,0.0232185,-0.0594512,-0.0187592,0.0130909,-0.0292405,0.0142983,0.0135293,0.0471098,0.0475464,-0.0417011,-0.0640614,-0.053297,-0.0689143,0.0879465,-0.0548818,0.0723397,0.141237,0.0319509,0.0506295,4.47196e-5,-0.0131204,0.00965925,0.0165248,0.00201354,0.0540104,0.0161022,-0.127756,-0.0706194
6,1,1,0,0,0,1,0,0,0.255046,0.544377,-0.269354,0.0545969,0.122965,-0.0845162,-0.105591,-0.19593,0.0594291,-0.0530593,0.0446624,0.0505941,0.0317026,0.066169,-0.0793738,-0.107617,0.0631714,-0.0116577,-0.012711,-0.0389265,-0.048534,0.027247,0.0088457,0.174406,-0.375456,-0.0739496,0.136094,0.0948738,-0.0190138,-0.04577,-0.20641,0.1

In [ ]:
# ------- TUNE -------  #
B_total = 68096.51 # set max total bids
B = 100 # max Avg. CPC in dataset is 94.69
N = 14229
# --------------------- #

n = 12 * 4 * 3 * num_keywords # months * regions * match * keywords
X = Matrix{Float64}(result)

    
model = Model(Gurobi.Optimizer)
set_silent(model)

@variable(model, b[1:n]>=0)
@variable(model, z[1:n], Bin)

@objective(model, Max,
    sum(X * conv_weights + conv_cpc_weight .* b .+ conv_const) -
    (X * clicks_weights + clicks_cpc_weight .* b .+ clicks_const)' * b
)
@constraint(model, sum(b) <= B_total)
@constraint(model, [i=1:n], b[i] >= -B * z[i])
@constraint(model, [i=1:n], b[i] <= B * z[i])
@constraint(model, sum(z) <= N)

optimize!(model);

In [ ]:
using Printf
@printf("ROI: %.0f\n", objective_value(model)) # 1,895,179

In [ ]:
b_vals = value.(b)
idxs = findall(x -> x > e, vals)

# Build a DataFrame of only active rows
bids = DataFrame(
    b_value = b_vals[idxs],      # keep the actual weight too
    keyword = keyword_df[vec(keyword_idx)[idxs], "Keyword"],
    month   = vec(month_col)[idxs],
    region  = vec(region_col)[idxs],
    match   = vec(match_col)[idxs],
)